# Utilizando resnet18 com os pesos pré-treinados do imagenet (ResNet18_Weights.IMAGENET1K_V1) para avaliar os datasets ImageNet e MNIST

## Carregar os datasets

### MNIST

In [1]:
import torchvision
from torchvision import transforms, datasets

transform_mnist = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.1307, 0.1307, 0.1307),
                         (0.3081, 0.3081, 0.3081))
])

mnist_test = datasets.MNIST(root='/home/storopoli/Downloads', train=False, transform=transform_mnist, download=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.7MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 509kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.63MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.33MB/s]


In [ ]:
#%pip install sympy==1.11.0

In [6]:
from torch.utils.data import DataLoader

batch_size=32

mnist_test_loader = DataLoader(dataset=mnist_test, batch_size=batch_size, shuffle=False)

### ImageNet

In [7]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# # baixar zip do site da stanford
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip -P /content/
print('Baixado!')

# # descompactar
!unzip -q /content/tiny-imagenet-200.zip -d /content/
print('Descompactado!')

# # Move para o drive
!mv /content/tiny-imagenet-200 /content/drive/MyDrive/tiny-imagenet-200
print('Movido para o drive!')

--2025-10-01 00:24:44--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.64.64
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs231n.stanford.edu/tiny-imagenet-200.zip [following]
--2025-10-01 00:24:44--  https://cs231n.stanford.edu/tiny-imagenet-200.zip
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘/content/tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  27.3MB/s    in 8.0s    

2025-10-01 00:24:52 (29.4 MB/s) - ‘/content/tiny-imagenet-200.zip’ saved [248100043/248100043]

^C


In [9]:
# Não entendo qual o problema que o !mv tem mas ele não esta copiando a pasta val
# O jeito é fazer manualmente
!rm -r /content/drive/MyDrive/tiny-imagenet-200/val
!mv /content/tiny-imagenet-200/val /content/drive/MyDrive/tiny-imagenet-200
print('Movido para o drive!')


rm: cannot remove '/content/drive/MyDrive/tiny-imagenet-200/val': No such file or directory
^C


In [10]:
import os
import shutil

val_dir = '/content/drive/MyDrive/tiny-imagenet-200/val/images'
val_anno = '/content/drive/MyDrive/tiny-imagenet-200/val/val_annotations.txt'
output_dir = '/content/drive/MyDrive/tiny-imagenet-200/val/'

# Lê as anotações e cria subpastas
with open(val_anno, 'r') as f:
    for line in f:
        parts = line.strip().split('\t')
        img, class_id = parts[0], parts[1]

        # Cria a pasta da classe se não existir
        class_dir = os.path.join(output_dir, class_id)
        os.makedirs(class_dir, exist_ok=True)

        # Move a imagem para a pasta certa
        img_src = os.path.join(val_dir, img)
        img_dst = os.path.join(class_dir, img)
        if os.path.exists(img_src):
            shutil.move(img_src, img_dst)

print("Val dataset reorganizado!")

Val dataset reorganizado!


In [12]:
from torchvision import datasets

transforms_imgnet = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

imgnet_test = datasets.ImageFolder(
    root='/content/tiny-imagenet-200/val',
    transform=transforms_imgnet
)

imgnet_test_loader = DataLoader(dataset=imgnet_test, batch_size=batch_size, shuffle=False)

## Criar a rede da ResNet18

In [13]:
from torchvision import models
import torch.nn as nn

# Carrega-la já pré-treinada

# MNIST
resnet_mnist = models.resnet18(weights="DEFAULT")
resnet_mnist.fc = nn.Linear(resnet_mnist.fc.in_features, 10)

# ImageNet
resnet_imgnet = models.resnet18(weights="DEFAULT")
resnet_imgnet.fc = nn.Linear(resnet_imgnet.fc.in_features, 200)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 179MB/s]


## GPU

In [14]:
import torch
torch.cuda.is_available()

True

## Validação

### MNIST

In [ ]:
resnet_mnist.eval() # coloca o modelo em modo de avaliação (sem calcular gradientes)

with torch.no_grad():
    correct = 0
    total = 0
    i = 0
    for images, labels in mnist_test_loader:
        if (i+1) % 50 == 0:
          print(f"Processado {i+1} batches...")

        # Feed-forward com as imagens de teste
        outputs = resnet_mnist(images)

        # gera predições usando a função max()
        _, predicted = torch.max(outputs.data, 1)

        # Acumula total e corretas
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        i+=1

    print(f"Acurácia do Modelo em 10K imagens de teste: {round((correct / total) * 100, 3)}")

Processado 50 batches...
Processado 100 batches...
Processado 150 batches...
Processado 200 batches...
Processado 250 batches...
Processado 300 batches...
Acurácia do Modelo em 10K imagens de teste: 8.23


### ImageNet

In [15]:
resnet_imgnet.eval() # coloca o modelo em modo de avaliação (sem calcular gradientes)

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in imgnet_test_loader:
        # Feed-forward com as imagens de teste
        outputs = resnet_imgnet(images)

        # gera predições usando a função max()
        _, predicted = torch.max(outputs.data, 1)

        # Acumula total e corretas
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Acurácia do Modelo em 10K imagens de teste: {round((correct / total) * 100, 3)}")

Acurácia do Modelo em 10K imagens de teste: 0.25
